In [ ]:
# ======================
# PNEUMONIA DETECTION SYSTEM
# WITH Grad-CAM VISUALIZATION
# ======================

# 1. SETUP AND DATA PREPARATION
# ------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Extract dataset
zip_path = "/content/drive/MyDrive/chest_xray.zip"
extract_path = "/content/chest_xray"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify paths
base_dir = '/content/chest_xray/chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# 2. DATA GENERATORS
# -------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# 3. MODEL BUILDING
# ------------------
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

# 4. TRAINING
# ------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,
    callbacks=callbacks
)

# 5. GRAD-CAM IMPLEMENTATION (NEW)
# ---------------------------------
def make_gradcam_heatmap(img_array, model, last_conv_layer_name="block_16_expand_relu"):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, preds = grad_model(img_array)
        loss = preds[:, 0]  # For binary classification

    grads = tape.gradient(loss, conv_outputs)[0]
    weights = tf.reduce_mean(grads, axis=(0, 1))
    heatmap = conv_outputs[0] @ weights[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap).numpy()
    return np.maximum(heatmap, 0) / np.max(heatmap)

def display_gradcam(img, heatmap, alpha=0.4):
    heatmap = np.uint8(255 * heatmap)
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    img = np.uint8(255 * img)
    superimposed_img = jet_heatmap * alpha + img * (1 - alpha)
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap='gray')
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img)
    plt.title("Grad-CAM")
    plt.axis('off')
    plt.show()

# 6. VISUALIZATION
# -----------------
# Get sample image
for images, labels in test_generator:
    sample_image = images[0]
    true_label = "Pneumonia" if labels[0] == 1 else "Normal"
    break

# Generate and display heatmap
heatmap = make_gradcam_heatmap(np.expand_dims(sample_image, axis=0), model)
display_gradcam(sample_image, heatmap)

# 7. EVALUATION
# --------------
y_pred = model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)

print(classification_report(test_generator.classes, y_pred_classes,
                           target_names=['Normal', 'Pneumonia']))

# Confusion Matrix
cm = confusion_matrix(test_generator.classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.show()